# Case Study 1 : Collecting Data from Twitter

Due Date: September 22, **before the beginning of class at 6:00pm**

* ------------

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/9f/Twitter_bird_logo_2012.svg/220px-Twitter_bird_logo_2012.svg.png">

**TEAM Members:**

    Matthew Beaulieu
    
    Yousef Fadila
    
    Meng Li
    
    Monica Tlachac
    
---

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

# Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [1]:
import pandas as pd
import matplotlib
import warnings
import json
import numpy as np
import wikiwords
from tweepy import Stream
from tweepy.streaming import StreamListener
import pymongo
import tweepy

warnings.simplefilter(action = "ignore", category = FutureWarning)
%matplotlib inline
#---------------------------------------------
# Define a Function to Login Twitter API
#
# XXX  - DON'T COMMIT TRUE CREDINTIALS TO GITHUB
#CONSUMER_KEY = 'k01qrrsDSOHdjUObbt1pWjhld'
#CONSUMER_SECRET ='FTBSa4UU5Nxaw00hNPJQvoAf60HinJM87GfQpuFM0YsVKDdoJD'
#OAUTH_TOKEN = '4229853605-mSgVKfVSZ9BSN7GbPgvUIPOT1Xl6F3nA2ynKMvy'
#OAUTH_TOKEN_SECRET = 'jyHtNVKGmqNw4HyBLMYG5i9BT3junKGVFnY6CE2tjJRFW'

# another set of auth to use when limit is hit.
CONSUMER_KEY = '7n45AnattFU5sb3MpbiwIoNYF'
CONSUMER_SECRET ='VtnEmlR99dG3hQ8FP8xb1TnomTAjSGkQYDJVhGa28IDPDEA2Cf'
OAUTH_TOKEN = '2497861306-W7d3EXegd4AY6jYt9ZaEz5rq5ZtguAAtp6KbseS'
OAUTH_TOKEN_SECRET = 'uv71S6pJwPibOFLA8WetoxWRb0nfqcQPK8f3ts30nEfQT'

# mango db auth.
MANGODB_SERVER = 'ds033046.mlab.com'
MANGODB_PORT = 33046
MANGODB_USER = 'ds501u'
MANGODB_PASS = 'ds501p'
MANGODB_NAME = 'ds501case1'
MANGODB_COLLECTION = 'tweets'
GEOBOX_USA = [-125,25.1,-60.5,49.1]

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

#----------------------------------------------

In [23]:
class MyListener(StreamListener):
    def __init__(self, max_count = 100):
        self.count = 0
        self.max_count = max_count
        db_client = pymongo.MongoClient(MANGODB_SERVER, MANGODB_PORT)
        db = db_client[MANGODB_NAME]
        db.authenticate(MANGODB_USER, MANGODB_PASS)      
        self.collection = db[MANGODB_COLLECTION]

    def on_data(self, data):
        try:
            if self.count >= self.max_count:
                return False
            # the question require output to file. so output to file and save in mangoDB
            with open('case1data.json', 'a') as f:
                f.write(data)
            self.count = self.count + 1
            datajson = json.loads(data)
            self.collection.insert_one(datajson)
            return True
        except BaseException as e:
            print(e)
        return True
 
    def on_error(self, status):
        print(status)
        return True
    
api = tweepy.API(auth)
limit = 1
twitter_stream = Stream(auth, MyListener(limit))
twitter_stream.filter(track=['@HillaryClinton'], languages=["en"], locations=GEOBOX_USA)
print("streaming done - limit: " + str(limit))


streaming done - limit: 1


In [187]:
# example how to get the state from the tweet.
lst = []
states = []
with open('hillary.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        lst.append(tweet)

states = [tweeet['place']['full_name'].split()
         for tweeet in lst if tweeet['place']]

states = [x[len(x) - 1] for x in states if x[(len(x) - 1)] != 'USA']      
states

['FL',
 'NM',
 'MA',
 'CA',
 'CA',
 'NJ',
 'FL',
 'HI',
 'NM',
 'NM',
 'CA',
 'NY',
 'MA',
 'NV']

In [58]:
lis = []
with open('hillary.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        lis.append(tweet) # load it as Python dict

with open('hillary2.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        lis.append(tweet) # load it as Python dict
    
df = pd.DataFrame(lis)

In [59]:
# we already filter english in streaming
# TODO ; use the function described in lecture to filter out english common words.
#df = df[df['lang'] == 'en']
text = df.text.str.lower()
for i,t in enumerate(text):
    new = ''.join(e for e in t if (e.isalnum() or e==' '))
    text[i] = new

In [60]:
text = text.str.split(' ')

In [61]:
word_dict = {}
for r in range(text.shape[0]):
    lis = text.values[r]
    for word in lis:
        if word in word_dict.keys():
            word_dict[word] +=1
        else:
            word_dict[word] = 1
word_dict

{'': 3380,
 'controversies': 1,
 'httpstconnjlnksrkr': 1,
 'cam1eddie': 1,
 'signs': 2,
 'standlincolnquots': 1,
 'cause': 19,
 'sending': 10,
 'whatu2014rich': 1,
 'bleacherbabble': 1,
 'httpstcolfwbiokwny': 1,
 'join': 21,
 'transition': 1,
 'significant': 2,
 'httpstcoqhhtzfx4c1': 1,
 'fierceness': 1,
 'cecilbadlands': 2,
 'httpstcoblgdg4ldfx': 1,
 'httpstcoxhpmagotpi': 1,
 'httpstcogdjywyohot': 1,
 'question.': 4,
 'escapade': 1,
 'poised!': 1,
 'heres': 35,
 'httpstcoa9ncjobqsk': 1,
 'analogy': 3,
 'texcountry44': 2,
 'httpstcotic9tka3m5': 1,
 'stan': 1,
 'hillswheels': 1,
 'mic': 9,
 'victims,': 2,
 'happier': 1,
 'thxmaga': 2,
 'still.': 1,
 'spawned': 2,
 'httpstcovwa40sib2u': 1,
 'lolololololol': 1,
 'bankrupt': 3,
 'judgenap': 1,
 'stevebutcher13': 1,
 'exitpollgate': 2,
 'media.': 1,
 'maamaw': 1,
 'httpstcoi4p18qusns': 1,
 'varneyco': 2,
 '\nthanks': 1,
 'game.': 3,
 '@voxday': 1,
 'suicide': 5,
 'thehighsign': 1,
 'job.': 2,
 'thanx': 2,
 'europesperance': 1,
 '317': 1,
 '

In [62]:
vals = []
for key in word_dict.keys():
    vals.append(word_dict.get(key))
    
tweet_df = pd.DataFrame({'words':list(word_dict.keys()), 'vals':vals})
tweet_df = tweet_df.sort_values('vals', ascending=False)
tweet_df['freq'] = tweet_df['words'].apply(wikiwords.freq)
tweet_df = tweet_df[tweet_df['freq'] > 1.0e-6]
tweet_df['ratio'] = tweet_df['vals'] / tweet_df['freq']

In [63]:
tweet_df.sort_values('ratio', ascending=False)

,vals,words,freq,ratio
13761,5837,rt,0.000008,7.000539e+08
9714,880,https,0.000003,3.083512e+08
3511,719,amp,0.000004,1.967924e+08
1906,778,trump,0.000006,1.311135e+08
1568,136,politico,0.000001,1.179679e+08
15633,540,hillary,0.000006,9.799528e+07
3465,142,poorest,0.000002,6.645896e+07
11760,67,benghazi,0.000001,5.300463e+07
4822,217,cant,0.000004,4.895326e+07
779,222,donors,0.000005,4.765627e+07


In [64]:
len(df)

9396

### Report some statistics about the tweets you collected 

* The topic of interest: < INSERT YOUR TOPIC HERE>


* The total number of tweets collected:  < INSERT THE NUMBER HERE>

In [ ]:
import folium
import pandas as pd
from IPython.display import HTML

def display(m, height=500):
    m.save("DS501map.html")
    get_html = m._repr_html_()
    s=get_html.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{0}" '
                 'style="width: 100%; height: {1}px; '
                 'border: none"></iframe>'.format(s, height))
    return embed
state_geo = r'us-states.json'
clinton_popularity_str = '{"WA": 7.8, "DE": 7.1, "WI": 6.8, "WV": 7.5, "HI": 5.4, "FL": 8.2, "WY": 5.1, "NH": 5.7, "NJ": 9.6, "NM": 6.8, "TX": 10, "LA": 5.9, "NC": 9.4, "ND": 3.2, "NE": 3.9, "TN": 7.8, "NY": 8.4, "PA": 8.0, "CA": 10.1, "NV": 10.3, "VA": 5.8, "CO": 7.7, "AK": 6.8, "AL": 7.1, "AR": 7.2, "VT": 5.0, "IL": 8.8, "GA": 8.8, "IN": 8.4, "IA": 5.1, "OK": 5.2, "AZ": 8.1, "ID": 6.6, "CT": 8.4, "ME": 7.2, "MD": 6.8, "MA": 6.7, "OH": 6.9, "UT": 5.5, "MO": 6.7, "MN": 5.6, "MI": 9.1, "RI": 10.1, "KS": 5.6, "MT": 5.8, "MS": 9.1, "SC": 8.8, "KY": 8.1, "OR": 8.5, "SD": 4.4}' 
clinton_popularity_json = json.loads(clinton_popularity_str)

map = folium.Map(location=(39, -100), zoom_start=4)
#map = folium.Map(location=[48, -102], zoom_start=3)
map.choropleth(geo_path=state_geo, data=clinton_popularity_json,
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.8, line_opacity=0.6,
             reset=False)
print ("the map should be shown below; if you can't see it, please find it in DS501map.html or re-run the cell")
display(map)

# Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary















**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary













**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary







* ------------------------

# Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [21]:
#definitios and config values

# As HillaryClinton has million of followers, we would use wait_on_rate_limit to fetch all of her friends. 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)
#api = tweepy.API(auth)

MANGODB_SERVER = 'ds035776.mlab.com'
MANGODB_PORT = 35776
MANGODB_CLINTON_FF_DB_NAME = "clinton_followers_friends"
MANGODB_CLINTON_FF_COLL = "collection"
db_clinton_ff_client = pymongo.MongoClient(MANGODB_SERVER, MANGODB_PORT)
db_clinton_ff = db_clinton_ff_client[MANGODB_CLINTON_FF_DB_NAME]
db_clinton_ff.authenticate(MANGODB_USER, MANGODB_PASS)  
 

True

In [ ]:
# fetch up to 200000
# to fetch all followers 
# call the API with 
#tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)
# and removethe .items(200000);

data_followers = {}
data_followers['type'] = 'followers'
data_followers['lst'] = []

for itm in tweepy.Cursor(api.followers_ids, id = 'HillaryClinton').items(200000):
    data_followers['lst'].append(itm)

# save the followers's list in mangoDB    
followers_json = json.loads(json.dumps(data_followers))
db_clinton_ff[MANGODB_CLINTON_FF_COLL].insert_one(followers_json)


data_friends = {}
data_friends['type'] = 'friends'
data_friends['lst'] = []

# fetch all friends list - as this list relativly samll we won't hit the limit here.

friends_ids = []     
for itm in tweepy.Cursor(api.friends_ids, id = 'HillaryClinton'):
    data_friends['lst'].append(itm)

# save the friend's list in mangoDB    
friends_json = json.loads(json.dumps(data_friends))  
db_clinton_ff[MANGODB_CLINTON_FF_COLL].insert_one(friends_json)



Rate limit reached. Sleeping for: 899


In [15]:
from prettytable import PrettyTable

# Given a set of user ids, this function calls get_screen_names and plots a table of the first (max_ids) ID's and screen names.
def draw_pretty_table(ids):
    users = api.lookup_users(user_ids=ids)
    pretty_table = PrettyTable(['ID','Screen Name'])
    [pretty_table.add_row ([row.id,row.screen_name]) for row in users]
    pretty_table.align = 'l'
    print(pretty_table)

In [16]:
# print 20 firends and 20 followers data
#
cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "friends"})
print ("\n20 friends of Hilary Clinton\n")
draw_pretty_table(cursor[0]['lst'][:20])

cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "followers"})
print ("\n20 followers of Hilary Clinton\n")
draw_pretty_table(cursor[0]['lst'][:20])
    


20 friends of Hilary Clinton

+--------------------+----------------+
| ID                 | Screen Name    |
+--------------------+----------------+
| 571202103          | Medium         |
| 21337440           | ChildDefender  |
| 23449384           | amberdiscko    |
| 128790234          | Samynemir      |
| 1656913327         | sarajacobs89   |
| 325886383          | SammyKoppelman |
| 802430450          | Natasha_S_Law  |
| 729761993461248000 | ktvibbs        |
| 115740215          | SarahAudelo    |
| 34782406           | Lincoln_Ross   |
| 3044781131         | HillaryforAR   |
| 113298560          | GunaRockYa     |
| 15972271           | CdotDukes      |
| 582037089          | MiguelAyala312 |
| 734768872625188864 | AndrewBatesNC  |
| 41021335           | TroyClair      |
| 4736170399         | BrianZuzenak   |
| 150885854          | SarahPeckVA    |
| 231673             | yianni         |
| 125083946          | GillDrummond   |
+--------------------+----------------+

20 follo

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [18]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "friends"})
friends = cursor[0]['lst']
cursor = db_clinton_ff[MANGODB_CLINTON_FF_COLL].find({"type": "followers"})
followers = cursor[0]['lst']

common_friends = set(friends).intersection(followers)

print (common_friends)

print ("\nMutual Friends\n")
draw_pretty_table(common_friends)


{4736170399}


*------------------------

# Problem 4: Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

In [ ]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary





*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through email to Prof. Paffenroth (rcpaffenroth@wpi.edu) *and* the TA Wen Liu (wliu3@wpi.edu).
        
** Note: Each team just needs to submits one submission **

# Grading Criteria:

** Totoal Points: 120 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

